In [11]:
!pip install h5py

In [12]:
pip install xarray h5netcdf h5py netcdf4



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install geopandas


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install rasterio


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
# ==========================
# 1. Imports et SSL
# ==========================
import ssl
import os
import warnings
import pandas as pd
import xarray as xr
import earthaccess
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

warnings.filterwarnings('ignore')
%matplotlib inline



In [22]:
# Deactivate SSL verification for Mac users 
import earthaccess, os

# Deactivate ssl in a fsspec session used by earthaccess
earthaccess.__store__.get_fsspec_session().kwargs['ssl'] = False
os.environ['FSSPEC_HTTP_VERIFY'] = 'False'

import certifi
import ssl
ssl_context = ssl.create_default_context(cafile=certifi.where())

In [17]:
# ==========================
# 2. Environment variables used for login
# ==========================
os.environ['EARTHDATA_USERNAME'] = 'tnourlyn'
os.environ['EARTHDATA_PASSWORD'] = 'Lutinette2411%'


In [18]:

# ==========================
# 3. EarthAccess authentification
# ==========================
try:
    auth = earthaccess.login(strategy="environment")
    print("✅ Authentification réussie !")
except Exception as e:
    print(f"❌ Échec de login : {e}")


✅ Authentification réussie !


In [19]:

# ==========================
# 4. Granules Search
# ==========================
minlon = 11.7 #Bouding box info for Nigeria -- To change depending on chosen country 
maxlon = 12.1
minlat = 13.0
maxlat = 13.4

granules = earthaccess.search_data(
    short_name='FLDAS_NOAH01_C_GL_M',
    temporal=("2020-01", "2024-12"),
    bounding_box=(minlon, minlat, maxlon, maxlat)
)
print(f"Granules found : {len(granules)}")


Granules found : 60


In [23]:

# ==========================
# 5. Data Opening
# ==========================
# Liste de fichiers locaux ou URLs
fn_list = earthaccess.open(granules)

ds = xr.open_dataset(fn_list[0], engine='h5netcdf')
print(ds)


QUEUEING TASKS | : 100%|██████████| 60/60 [00:00<00:00, 6880.04it/s]
PROCESSING TASKS | : 100%|██████████| 60/60 [00:23<00:00,  2.55it/s]
COLLECTING RESULTS | : 100%|██████████| 60/60 [00:00<00:00, 391991.03it/s]


<xarray.Dataset> Size: 605MB
Dimensions:                 (time: 1, bnds: 2, Y: 1500, X: 3600)
Coordinates:
  * time                    (time) datetime64[ns] 8B 2020-01-01
  * Y                       (Y) float64 12kB -59.95 -59.85 ... 89.85 89.95
  * X                       (X) float64 29kB -179.9 -179.8 ... 179.9 180.0
Dimensions without coordinates: bnds
Data variables: (12/29)
    time_bnds               (time, bnds) datetime64[ns] 16B ...
    Evap_tavg               (time, Y, X) float32 22MB ...
    LWdown_f_tavg           (time, Y, X) float32 22MB ...
    Lwnet_tavg              (time, Y, X) float32 22MB ...
    Psurf_f_tavg            (time, Y, X) float32 22MB ...
    Qair_f_tavg             (time, Y, X) float32 22MB ...
    ...                      ...
    SoilMoi40_100cm_tavg    (time, Y, X) float32 22MB ...
    SoilMoi100_200cm_tavg   (time, Y, X) float32 22MB ...
    SoilTemp00_10cm_tavg    (time, Y, X) float32 22MB ...
    SoilTemp10_40cm_tavg    (time, Y, X) float32 22MB ...

Let's create the data frame thanks to the extracted data 

In [24]:
#Let's create a list of variables : 
varNames =[
    'Evap_tavg',
    'LWdown_f_tavg',
    'Lwnet_tavg',
    'Psurf_f_tavg',
    'Qair_f_tavg',
    'Qg_tavg',
    'Qh_tavg',
    'Qle_tavg',
    'Qs_tavg',
    'Qsb_tavg',
    'RadT_tavg',
    'Rainf_f_tavg',
    'SWE_inst',
    'SWdown_f_tavg',
    'SnowCover_inst',
    'SnowDepth_inst',
    'Snowf_tavg',
    'Swnet_tavg',
    'Tair_f_tavg',
    'Wind_f_tavg',
    'SoilMoi00_10cm_tavg',
    'SoilMoi10_40cm_tavg',
    'SoilMoi40_100cm_tavg',
    'SoilMoi100_200cm_tavg',
    'SoilTemp00_10cm_tavg',
    'SoilTemp10_40cm_tavg',
    'SoilTemp40_100cm_tavg',
    'SoilTemp100_200cm_tavg'
]

In [30]:
#We select the most important variables :

ds_sel = ds[varNames].where(
    (ds.X >= minlon) & (ds.X <= maxlon) &
    (ds.Y >= minlat) & (ds.Y <= maxlat),
    drop=True 
)
df_NASA = ds_sel.to_dataframe().reset_index()



In [32]:
df_NASA.head()

,time,Y,X,Evap_tavg,LWdown_f_tavg,Lwnet_tavg,Psurf_f_tavg,Qair_f_tavg,Qg_tavg,Qh_tavg,...,Tair_f_tavg,Wind_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi40_100cm_tavg,SoilMoi100_200cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp40_100cm_tavg,SoilTemp100_200cm_tavg
0,2020-01-01,13.05,11.75,0.000004,301.766235,-114.224480,97730.023438,0.003047,-2.437553,36.012638,...,293.387177,8.211732,0.146190,0.280817,0.287616,0.316367,294.138306,294.960815,296.088745,297.725525
1,2020-01-01,13.05,11.85,0.000004,301.833221,-114.142052,97748.367188,0.003052,-2.515010,35.716026,...,293.406555,8.223705,0.146113,0.292110,0.299296,0.326776,294.156158,295.001312,296.138977,297.780273
2,2020-01-01,13.05,11.95,0.000004,301.819672,-114.200279,97717.820312,0.003048,-2.598254,35.661167,...,293.433441,8.227583,0.146031,0.307631,0.313970,0.336462,294.186920,295.054352,296.193878,297.826233
3,2020-01-01,13.05,12.05,0.000004,302.188110,-114.435616,97798.843750,0.003052,-2.563051,36.731537,...,293.535858,8.229089,0.145911,0.291901,0.298342,0.323498,294.303925,295.155914,296.303986,297.942688
4,2020-01-01,13.15,11.75,0.000004,301.254425,-114.119308,97715.625000,0.003028,-2.477718,36.182880,...,293.261108,8.256189,0.146109,0.271670,0.276233,0.297835,294.013611,294.844482,296.001343,297.675964


In [31]:
df_NASA.isna().sum()

time                      0
Y                         0
X                         0
Evap_tavg                 0
LWdown_f_tavg             0
Lwnet_tavg                0
Psurf_f_tavg              0
Qair_f_tavg               0
Qg_tavg                   0
Qh_tavg                   0
Qle_tavg                  0
Qs_tavg                   0
Qsb_tavg                  0
RadT_tavg                 0
Rainf_f_tavg              0
SWE_inst                  0
SWdown_f_tavg             0
SnowCover_inst            0
SnowDepth_inst            0
Snowf_tavg                0
Swnet_tavg                0
Tair_f_tavg               0
Wind_f_tavg               0
SoilMoi00_10cm_tavg       0
SoilMoi10_40cm_tavg       0
SoilMoi40_100cm_tavg      0
SoilMoi100_200cm_tavg     0
SoilTemp00_10cm_tavg      0
SoilTemp10_40cm_tavg      0
SoilTemp40_100cm_tavg     0
SoilTemp100_200cm_tavg    0
dtype: int64